In general, there are three types of constraints:
- Constraint repair
- Penalty methods
- Feasibility-preserving operations

From an implementation standpoint, constraint repair takes the updated particles at time $t$ and applies some function that modifies the particle positions to be enforce feasibility. Penalty methods instead modify the computed objective score after the particle position update as a function of the new positions and the constraints. Finally, feasibility-preserving operations overload the algebraic operations (addition and multiplication) to guarantee feasibility (e.g., using the Aitchison geometry to enforce sum-to-one constraints). 

In [2]:
include("./src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, PSO, Objectives, Designs
using LinearAlgebra
using NPZ
using CDDLib
using Polyhedra
using Base.Iterators

In [3]:
function unit_hypercube_constraints(d::Int)
    I_d = Matrix{Float64}(I, d, d)
    A = vcat(I_d, -I_d)
    b = vcat(ones(d), zeros(d))
    return A, b
end

function hypercube_constraints(n::Int, k::Real)
    I_n = Matrix{Float64}(I, n, n)
    A = vcat(I_n, -I_n)
    b = vcat(fill(Float64(k), n), fill(Float64(k), n))
    return A, b
end


hypercube_constraints (generic function with 1 method)

In [4]:
# experiment = Experiments.with_factor_ratio(experiment, 2, 3, 2.0)
# experiment = Experiments.with_factor_ratio(experiment, 1, 3, 2.0)
# A = copy(experiment.constraints.A)
# b = Float64.([100, 100, 100, 100, 100, 100, 0, 0])

In [5]:
N = 100
K = 100
bound = 100

A, b = hypercube_constraints(N, bound)
experiment = Experiments.create(N, K)
experiment = Experiments.with_linear_constraints(experiment, A, b)
for i in 1:(N-1)
    experiment = Experiments.with_factor_ratio(experiment, i, N, 2.0)
end

In [6]:
function save_history(history::Vector; location = "pso_griewank.npy")
    velocities = cat([h.state.velocities for h in history]...; dims=2)
    velocities = permutedims(velocities, (2, 1, 3))

    positions = cat([h.state.positions for h in history]...; dims=2)
    positions = permutedims(positions, (2, 1, 3))

    scores = cat([h.fitness.scores for h in history]...; dims=2)'
    npzwrite(location, Dict("positions" => positions, "velocities" => velocities, "scores" => scores))
end

function save_history_3d(history::Vector; location = "pso_griewank.npy")
    velocities = cat([h.state.velocities for h in history]...; dims=4)
    velocities = permutedims(velocities, (4, 1, 2, 3))

    positions = cat([h.state.positions for h in history]...; dims=4)
    positions = permutedims(positions, (4, 1, 2, 3))

    scores = cat([h.fitness.scores for h in history]...; dims=2)'
    npzwrite(location, Dict("positions" => positions, "velocities" => velocities, "scores" => scores))
end

save_history_3d (generic function with 1 method)

In [7]:
runner_params = PSO.runner_params(200, 200, 1e-6)
pso_params = PSO.create_hyperparams(100)

Main.MetaDoE.PSO.HyperParams(100, 0.7213475204444817, 1.1931471805599454, 1.1931471805599454, 3)

In [17]:
objectives = [Objectives.rastrigin]
# objectives = [Objectives.griewank, Objectives.ackley, Objectives.rastrigin, Objectives.rosenbrock]
enforcers = [ConstraintEnforcement.Linear]

for (obj, enf) in Iterators.product(objectives, enforcers)
    println("Objective: $obj, Enforcer: $enf")
    context = PSO.create_context(
        experiment, 
        obj; 
        callback = PSO.aggregate_results(;save_world=true), 
        runner_params=runner_params,
        enforcer_type = enf
    )
    runner_state, history = PSO.optimize(context)
    println("Best score: $(runner_state.swarm.memory.global_best_score)")
    save_history_3d(history; location = "100_constrained_$(obj)_$enf.npy")
end

Objective: rastrigin, Enforcer: Linear
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
299 rows, 101 cols, 697 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-299); columns 0(-101); elements 0(-697) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value    

In [9]:
# get_vertices = (p) -> collect(points(p))

# p = polyhedron(hrep(A, b), CDDLib.Library())
# verts = get_vertices(p)
# verts = hcat(verts...)'
# npzwrite("verts.npy", verts)

In [10]:
# using .MetaDoE: TensorOps

In [11]:
# npzwrite("sample.npy", TensorOps.squeeze(initializer(10_000)))

In [12]:
# TensorOps.squeeze(initializer(20))